In [19]:
from langchain.document_loaders.base import Document
from langchain.indexes import VectorstoreIndexCreator
from langchain.utilities import ApifyWrapper
from serpapi import GoogleSearch
import os

from langchain.llms import OpenAI

In [25]:
llm = OpenAI(
    model="text-davinci-003",
    temperature=0,
)

apify = ApifyWrapper()

In [37]:
params = {
    "q": "wandb features",
    "api_key": os.environ["SERPAPI_API_KEY"],
}

search = GoogleSearch(params)
results = search.get_dict()
organic_results = results["organic_results"]
link = organic_results[0]['link']

In [38]:
link

'https://docs.wandb.ai/guides/app/features'

In [39]:
loader = apify.call_actor(
    actor_id="apify/website-content-crawler",
    run_input={"startUrls": [{"url": link}]},
    dataset_mapping_function=lambda item: Document(
        page_content=item["text"] or "", metadata={"source": item["url"]}
    ),
)

index = VectorstoreIndexCreator().from_loaders([loader])

In [40]:
query = "What are some features of wandb?"
result = index.query_with_sources(query, llm=llm)

print(result["answer"])
print(result["sources"])

 Features of wandb include Panels, Custom Charts, Runs table, Tags, Notes, Teams, Organizations, System Metrics, Anonymous Mode, Secure Storage Connector, Team Profile, and logging results from a training loop.

https://docs.wandb.ai/guides/app/features, https://docs.wandb.ai/guides/app/features/teams, https://docs.wandb.ai/guides/app/features/anon
